# Лабораторная работа №5
## Выполнил студент группы ФИО ГРУППА

### Оглавление
1. [Задание 1](#Задание-№1)
2. [Задание 2](#Задание-№2)
4. [Вывод](#Вывод)

> Дополнительные модули, использованные при выполнение лабораторной

In [ ]:
from datetime import datetime as t
from collections import defaultdict
from collections import deque
import ipywidgets as widgets
from IPython.display import display, Markdown

### Задание №1
Реализовать методы поиска подстроки в строке. Добавить возможность ввода строки и подстроки с клавиатуры. Предусмотреть возможность существования пробела. Реализовать возможность выбора опции чувствительности или нечувствительности к регистру. Оценить время работы каждого алгоритма поиска и сравнить его со временем работы стандартной функции поиска, используемой в выбранном языке программирования.

#### Алгоритм Кнута-Морриса-Пратта

In [1]:
from datetime import datetime as t
import ipywidgets as widgets
from IPython.display import display, clear_output

def build_kmp_prefix(pattern):
    prefix = [0] * len(pattern)
    j = 0
    for i in range(1, len(pattern)):
        while j > 0 and pattern[i] != pattern[j]:
            j = prefix[j - 1]
        if pattern[i] == pattern[j]:
            j += 1
        prefix[i] = j
    return prefix

def kmp_search(text, pattern, case_sensitive=True):
    if not case_sensitive:
        text = text.lower()
        pattern = pattern.lower()

    prefix = build_kmp_prefix(pattern)
    result = []
    j = 0
    for i in range(len(text)):
        while j > 0 and text[i] != pattern[j]:
            j = prefix[j - 1]
        if text[i] == pattern[j]:
            j += 1
        if j == len(pattern):
            result.append(i - j + 1)
            j = prefix[j - 1]
    return result

text_input = widgets.Textarea(description="Строка", layout=widgets.Layout(width='100%'))
pattern_input = widgets.Text(description="Подстрока")
case_toggle = widgets.Checkbox(value=True, description="Учитывать регистр")
run_button = widgets.Button(description="Поиск", button_style='success')
output = widgets.Output()

def run_search(b):
    with output:
        clear_output()
        text = text_input.value
        pattern = pattern_input.value
        case = case_toggle.value

        if not text or not pattern:
            print("⚠️ Пожалуйста, введите строку и подстроку.")
            return

        start_kmp = t.now()
        result_kmp = kmp_search(text, pattern, case_sensitive=case)
        duration_kmp = (t.now() - start_kmp).total_seconds()

        if not case:
            text_proc = text.lower()
            pattern_proc = pattern.lower()
        else:
            text_proc = text
            pattern_proc = pattern

        start_std = t.now()
        result_std = []
        pos = text_proc.find(pattern_proc)
        while pos != -1:
            result_std.append(pos)
            pos = text_proc.find(pattern_proc, pos + 1)
        duration_std = (t.now() - start_std).total_seconds()

        print("Результаты KMP:")
        print("Найдено в позициях:", result_kmp)
        print("Время выполнения:", f"{duration_kmp:.8f} сек\n")

        print("Результаты стандартного str.find():")
        print("Найдено в позициях:", result_std)
        print("Время выполнения:", f"{duration_std:.8f} сек")

run_button.on_click(run_search)

display(widgets.VBox([text_input, pattern_input, case_toggle, run_button, output]))

#### Упрощенный алгоритм Бойера-Мура

In [2]:
from datetime import datetime as t
import ipywidgets as widgets
from IPython.display import display, clear_output
from collections import defaultdict

def build_bad_char_table(pattern):
    table = defaultdict(lambda: -1)
    for i, char in enumerate(pattern):
        table[char] = i
    return table

def boyer_moore_search(text, pattern, case_sensitive=True):
    if not case_sensitive:
        text = text.lower()
        pattern = pattern.lower()

    m = len(pattern)
    n = len(text)
    if m == 0:
        return []

    table = build_bad_char_table(pattern)
    result = []

    s = 0
    while s <= n - m:
        j = m - 1
        while j >= 0 and pattern[j] == text[s + j]:
            j -= 1
        if j < 0:
            result.append(s)
            s += m if s + m < n else 1
        else:
            shift = max(1, j - table[text[s + j]])
            s += shift
    return result

text_input_bm = widgets.Textarea(description="Строка", layout=widgets.Layout(width='100%'))
pattern_input_bm = widgets.Text(description="Подстрока")
case_toggle_bm = widgets.Checkbox(value=True, description="Учитывать регистр")
run_button_bm = widgets.Button(description="Поиск BM", button_style='info')
output_bm = widgets.Output()

def run_bm_search(b):
    with output_bm:
        clear_output()
        text = text_input_bm.value
        pattern = pattern_input_bm.value
        case = case_toggle_bm.value
        start_bm = t.now()
        result_bm = boyer_moore_search(text, pattern, case_sensitive=case)
        duration_bm = (t.now() - start_bm).total_seconds()

        start_std = t.now()
        result_std = []
        i = text.find(pattern) if case else text.lower().find(pattern.lower())
        while i != -1:
            result_std.append(i)
            i = text.find(pattern, i + 1) if case else text.lower().find(pattern.lower(), i + 1)
        duration_std = (t.now() - start_std).total_seconds()

        print("Результаты Бойера-Мура:")
        print("Найдено в позициях:", result_bm)
        print("Время выполнения BM:", duration_bm, "сек")
        print("Результаты стандартной функции поиска:")
        print("Найдено в позициях:", result_std)
        print("Время выполнения стандартной:", duration_std, "сек")

run_button_bm.on_click(run_bm_search)

display(widgets.VBox([text_input_bm, pattern_input_bm, case_toggle_bm, run_button_bm, output_bm]))

### Задание №2
Написать программу, определяющую, является ли данное
расположение «решаемым», то есть можно ли из него за конечное число
шагов перейти к правильному. Если это возможно, то необходимо найти хотя
бы одно решение - последовательность движений, после которой числа будут
расположены в правильном порядке.
#### Входные данные: массив чисел, представляющий собой расстановку в
Порядке «слева направо, сверху вниз». Число 0 обозначает пустое поле.
Например, массив [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 0] представляет
собой «решенную» позицию элементов.
#### Выходные данные: если решения нет, то функция должна вернуть
Пустой массив []. Если решение есть, то необходимо представить решение —
для каждого шага записывается номер передвигаемого на данном шаге
элемента. 

In [3]:
from collections import deque
import ipywidgets as widgets
from IPython.display import display, clear_output

def is_solvable(puzzle):
    inv_count = 0
    for i in range(15):
        for j in range(i + 1, 16):
            if puzzle[i] != 0 and puzzle[j] != 0 and puzzle[i] > puzzle[j]:
                inv_count += 1
    zero_row = 3 - (puzzle.index(0) // 4) 
    return (inv_count + zero_row) % 2 == 0

def get_neighbors(state):
    moves = []
    zero = state.index(0)
    row, col = zero // 4, zero % 4
    directions = [(-1, 0), (1, 0), (0, -1), (0, 1)] 
    for dr, dc in directions:
        nr, nc = row + dr, col + dc
        if 0 <= nr < 4 and 0 <= nc < 4:
            new_pos = nr * 4 + nc
            new_state = list(state)
            new_state[zero], new_state[new_pos] = new_state[new_pos], new_state[zero]
            moves.append((tuple(new_state), new_state[zero]))
    return moves

def solve_puzzle(start):
    if not is_solvable(start):
        return []

    goal = tuple(range(1, 16)) + (0,)
    queue = deque()
    visited = set()
    parent = {}
    move = {}

    queue.append(tuple(start))
    visited.add(tuple(start))

    while queue:
        current = queue.popleft()
        if current == goal:
            path = []
            while current in move:
                path.append(move[current])
                current = parent[current]
            return path[::-1]  
        for neighbor, moved_tile in get_neighbors(current):
            if neighbor not in visited:
                visited.add(neighbor)
                parent[neighbor] = current
                move[neighbor] = moved_tile
                queue.append(neighbor)
    return []

input_text = widgets.Text(
    value='1,2,3,4,5,6,7,8,9,10,11,12,13,0,14,15',
    description='Ввод:',
    layout=widgets.Layout(width='100%')
)
run_button = widgets.Button(description="Решить", button_style='success')
output = widgets.Output()

def run_solver(b):
    with output:
        clear_output()
        try:
            numbers = list(map(int, input_text.value.strip().split(',')))
            if len(numbers) != 16 or set(numbers) != set(range(16)):
                print("Ошибка: введите 16 уникальных чисел от 0 до 15.")
                return
            result = solve_puzzle(numbers)
            if result:
                print("Позиция решаема.")
                print("Последовательность перемещений:", result)
                print("Количество шагов:", len(result))
            else:
                print("Позиция не решаема.")
        except Exception as e:
            print("Ошибка:", e)

run_button.on_click(run_solver)

display(widgets.VBox([input_text, run_button, output]))

### Вывод